In [3]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
#from keras.utils.visualize_util import plot
import numpy as np


Using TensorFlow backend.


In [4]:
fin = open("data/11-0.txt",'rb')
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("ascii","ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

In [5]:
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [6]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


In [7]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
        y[i ,char2index[label_chars[i]]] = 1
        

In [8]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATIONS = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, input_shape=(SEQLEN, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


In [9]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" %(iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATIONS)
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" %(test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # MOVE FORWARD WITH TEST_CHARS + YPRED
        test_chars  =  test_chars[1:] + ypred
print()

Iteration #: 0
Epoch 1/1
158773/158773 [==============================] - 10s 61us/step - loss: 2.3345
Generating from seed: like a fro
like a frould and alice and alice and alice and alice and alice and alice and alice and alice and alice and al==================================================
Iteration #: 1
Epoch 1/1
158773/158773 [==============================] - 7s 42us/step - loss: 2.0557
Generating from seed: igh: then 
igh: then the the her and the said the har her and the said the har her and the said the har her and the said ==================================================
Iteration #: 2
Epoch 1/1
158773/158773 [==============================] - 6s 38us/step - loss: 1.9514
Generating from seed: sure shes 
sure shes all the said the could the could the could the could the could the could the could the could the cou==================================================
Iteration #: 3
Epoch 1/1
158773/158773 [==============================] - 6s 40us/step - loss: 1.8690
Generatin

# TESTING ON RAW TEXT FILE OF ALICE IN WONDERLAND (CONATINS PUBLISHER INFO, LISCENCES , GARBAGE)
# AT 25 NUM_ITERATIONS GETTING LOSS - 1.
# AT 35 NUM_ITERATIONS GETTING LOSS - 1.3553
# AT 50 NUM_ITERATIONS GETTING LOSS - 1.3074
# AT 100 NUM_ITERATIONS GETTING LOSS - 1.2519 
# (eg:Generating from seed:  be collec
# be collected round on the project gutenberg-tm electronic works in a hard, and said to herself, in a could to)